## Implementing binary decision trees
The goal of this notebook is to implement your own binary decision tree classifier. You will:

Use SFrames to do some feature engineering. Transform categorical variables into binary variables. Write a function to compute the number of misclassified examples in an intermediate node. Write a function to find the best feature to split on. Build a binary decision tree from scratch. Make predictions using the decision tree. Evaluate the accuracy of the decision tree. Visualize the decision at the root node.

In [1]:
import pandas as pd
import numpy as np
import json

def readjson(path):
    try:
        with open(path,'r') as f:
            data = json.load(f)
        return data
    except FileNotFoundError:
        raise FileNotFoundError( "Please check the path and try agaian!!!")

In [2]:
loans = pd.read_csv('lending-club-data.csv')

/anaconda3/envs/AI-ML/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans = loans.drop('bad_loans', axis = 1)

In [4]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length'          # number of years of employment
           ]
target = 'safe_loans'

In [5]:
loans = loans[features + [target]]

In [6]:
loans.fillna('n/a')
loans.reset_index(drop=True, inplace=True)
for i in range(len(loans)):
    for j in loans.columns:
        x = loans.at[i, j]
        if isinstance(x, str):
            x = x.lower().strip()
            loans.at[i, j] = x
        else:
            continue
    
    
loans = pd.get_dummies(loans)

In [7]:
train_idx = readjson('module-5-assignment-2-train-idx.json')
test_idx = readjson('module-5-assignment-2-test-idx.json')

In [9]:
train_data = loans.iloc[train_idx].copy(deep=True)
test_data = loans.iloc[test_idx].copy(deep=True)

### Decision tree implementation

In this section, we will implement binary decision trees from scratch. There are several steps involved in building a decision tree. For that reason, we have split the entire assignment into several sections.

### Function to count number of mistakes while predicting majority class

Recall from the lecture that prediction at an intermediate node works by predicting the majority class for all data points that belong to this node. Now, we will write a function that calculates the number of misclassified examples when predicting the majority class. This will be used to help determine which feature is the best to split on at a given node of the tree.

Note: Keep in mind that in order to compute the number of mistakes for a majority classifier, we only need the label (y values) of the data points in the node.

Steps to follow:

Steps to follow :

Step 1: Calculate the number of safe loans and risky loans.

Step 2: Since we are assuming majority class prediction, all the data points that are not in the majority class are considered mistakes.

Step 3: Return the number of mistakes.

Now, let us write the function intermediate_node_num_mistakes which computes the number of misclassified examples of an intermediate node given the set of labels (y values) of the data points contained in the node. Fill in the places where you find ## YOUR CODE HERE. There are three places in this function for you to fill in.

In [10]:
def intermediate_node_num_mistakes(labels_in_node):
    # Corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0
    
    # Count the number of 1's (safe loans)
    num_of_positive = (labels_in_node == +1).sum()
    
    # Count the number of -1's (risky loans)
    num_of_negative = (labels_in_node == -1).sum()
                
    # Return the number of mistakes that the majority classifier makes.
    return num_of_negative if num_of_positive > num_of_negative else num_of_positive

Because there are several steps in this assignment, we have introduced some stopping points where you can check your code and make sure it is correct before proceeding. To test your intermediate_node_num_mistakes function, run the following code until you get a Test passed!, then you should proceed. Otherwise, you should spend some time figuring out where things went wrong

In [12]:
example_labels = np.array([-1, -1, 1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print ('Test passed!')
else:
    print ('Test 1 failed... try again!')

# Test case 2
example_labels = np.array([-1, -1, 1, 1, 1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print ('Test passed!')
else:
    print ('Test 2 failed... try again!')
    
# Test case 3
example_labels = np.array([-1, -1, -1, -1, -1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print ('Test passed!')
else:
    print ('Test 3 failed... try again!')

Test passed!
Test passed!
Test passed!


### Function to pick best feature to split on
The function best_splitting_feature takes 3 arguments:

The data (DataFrame of data which includes all of the feature columns and label column)
The features to consider for splits (a list of strings of column names to consider for splits)
The name of the target/label column (string)
The function will loop through the list of possible features, and consider splitting on each of them. It will calculate the classification error of each split and return the feature that had the smallest classification error when split on.

Recall that the classification error is defined as follows:

classification error= #mistakes ÷ #total examples
Follow these steps:

Step 1: Loop over each feature in the feature list

Step 2: Within the loop, split the data into two groups: one group where all of the data has feature value 0 or False (we will call this the left split), and one group where all of the data has feature value 1 or True (we will call this the right split). Make sure the left split corresponds with 0 and the right split corresponds with 1 to ensure your implementation fits with our implementation of the tree building process.

Step 3: Calculate the number of misclassified examples in both groups of data and use the above formula to compute the classification error.

Step 4: If the computed error is smaller than the best error found so far, store this feature and its error.
This may seem like a lot, but we have provided pseudocode in the comments in order to help you implement the function correctly.

Note: Remember that since we are only dealing with binary features, we do not have to consider thresholds for real-valued features. This makes the implementation of this function much easier.

Fill in the places where you find ## YOUR CODE HERE. There are five places in this function for you to fill in.

In [13]:
def best_splitting_feature(data, features, target):
    
    best_feature = None           # Keep track of the best feature 
    best_error = float('inf')     # Keep track of the best error so far 
    # Note: Since error is always <= 1, we should intialize it with something larger than 1.

    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(data))  
    
    # Loop through each feature to consider splitting on that feature
    for feature in features:
        
        # The left split will have all data points where the feature value is 0
        left_split = data[data[feature] == 0].copy(deep=True)
        
        # The right split will have all data points where the feature value is 1
        right_split =  data[data[feature] == 1].copy(deep=True)
            
        # Calculate the number of misclassified examples in the left split.
        # Remember that we implemented a function for this! (It was called intermediate_node_num_mistakes)
        left_mistakes = intermediate_node_num_mistakes(left_split[target].values)            

        # Calculate the number of misclassified examples in the right split.
        right_mistakes = intermediate_node_num_mistakes(right_split[target].values)
            
        # Compute the classification error of this split.
        # Error = (# of mistakes (left) + # of mistakes (right)) / (# of data points)
        error = (left_mistakes + right_mistakes) / num_data_points

        # If this is the best error we have found so far, store the feature as best_feature and the error as best_error
        if error < best_error:
            best_feature = feature            
            best_error = error              
    
    return best_feature # Return the best feature we found

To test your best_splitting_feature function, run the following code:

In [14]:
train_data.columns

Index(['safe_loans', 'grade_a', 'grade_b', 'grade_c', 'grade_d', 'grade_e',
       'grade_f', 'grade_g', 'term_36 months', 'term_60 months',
       'home_ownership_mortgage', 'home_ownership_other', 'home_ownership_own',
       'home_ownership_rent', 'emp_length_1 year', 'emp_length_10+ years',
       'emp_length_2 years', 'emp_length_3 years', 'emp_length_4 years',
       'emp_length_5 years', 'emp_length_6 years', 'emp_length_7 years',
       'emp_length_8 years', 'emp_length_9 years', 'emp_length_< 1 year'],
      dtype='object')

In [19]:
features = [i for i in train_data.columns.tolist() if i!='safe_loans']

In [20]:
if best_splitting_feature(train_data, features, 'safe_loans') == 'term_36 months':
    print ('Test passed!')
else:
    print ('Test failed... try again!')

Test passed!


### Building the tree
With the above functions implemented correctly, we are now ready to build our decision tree. Each node in the decision tree is represented as a dictionary which contains the following keys and possible values:

{ 
   'is_leaf'            : True/False.
   
   'prediction'         : Prediction at the leaf node.
   
   'left'               : (dictionary corresponding to the left tree).
   
   'right'              : (dictionary corresponding to the right tree).
   
   'splitting_feature'  : The feature that this node splits on.
   
}

First, we will write a function that creates a leaf node given a set of target values. Fill in the places where you find ## YOUR CODE HERE. There are three places in this function for you to fill in.

In [21]:
def create_leaf(target_values):
    
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf':  True  }
    
    # Count the number of data points that are +1 and -1 in this node.
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])
    
    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] = +1
    else:
        leaf['prediction'] =  -1
        
    # Return the leaf node        
    return leaf

A function that learns the decision tree recursively and implements 3 stopping conditions:

Stopping condition 1: All data points in a node are from the same class.
<br/>Stopping condition 2: No more features to split on.
<br/>Additional stopping condition: In addition to the above two stopping conditions covered in lecture, in this assignment we will also consider a stopping condition based on the max_depth of the tree. By not letting the tree grow too deep, we will save computational effort in the learning process.
<br/>Now, we will write down the skeleton of the learning algorithm. Fill in the places where you find ## YOUR CODE HERE. There are seven places in this function for you to fill in.

In [26]:
def decision_tree_create(data, features, target, current_depth = 0, max_depth = 10):
    remaining_features = features[:].copy() # Make a copy of the features.
    
    target_values = data[target].values
    print ("--------------------------------------------------------------------")
    print ("Subtree, depth = %s (%s data points)." % (current_depth, len(target_values)))
    

    # Stopping condition 1
    # (Check if there are mistakes at current node.
    # Recall you wrote a function intermediate_node_num_mistakes to compute this.)
    if intermediate_node_num_mistakes(target_values) == 0:
        print ("Stopping condition 1 reached." )    
        # If not mistakes at current node, make current node a leaf node
        return create_leaf(target_values)
    
    # Stopping condition 2 (check if there are remaining features to consider splitting on)
    if len(remaining_features) == 0:
        print ("Stopping condition 2 reached." )   
        # If there are no remaining features to consider, make current node a leaf node
        return create_leaf(target_values)    
    
    # Additional stopping condition (limit tree depth)
    if current_depth >= max_depth:  ## YOUR CODE HERE
        print ("Reached maximum depth. Stopping for now.")
        # If the max tree depth has been reached, make current node a leaf node
        return create_leaf(target_values)

    # Find the best splitting feature (recall the function best_splitting_feature implemented above)
    splitting_feature = best_splitting_feature(data, features, target)
    
    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0].copy(deep=True)
    right_split = data[data[splitting_feature] == 1].copy(deep=True)
    remaining_features.remove(splitting_feature)
    print ("Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split)))
    
    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        print ("Creating leaf node.")
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        print ("Creating leaf node.")
        return create_leaf(right_split[target])

        
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, current_depth + 1, max_depth)        
    ## YOUR CODE HERE
    right_tree = decision_tree_create(right_split, remaining_features, target, current_depth + 1, max_depth) 

    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

Here is a recursive function to count the nodes in your tree:

In [27]:
def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return 1 + count_nodes(tree['left']) + count_nodes(tree['right'])

Run the following test code to check your implementation. Make sure you get 'Test passed' before proceeding.

In [28]:
small_data_decision_tree = decision_tree_create(train_data, features, 'safe_loans', max_depth = 3)
if count_nodes(small_data_decision_tree) == 13:
    print ('Test passed!')
else:
    print ('Test failed... try again!')
    print ('Number of nodes found                :', count_nodes(small_data_decision_tree))
    print ('Number of nodes that should be there : 13') 

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_a. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_b. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 3 (1048 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Split on feature emp_length_< 1 year. (90, 11)
--------------------------------------------------------------------
Subtree, depth = 3 (90 data po

In [29]:
small_data_decision_tree

{'is_leaf': False,
 'prediction': None,
 'splitting_feature': 'term_36 months',
 'left': {'is_leaf': False,
  'prediction': None,
  'splitting_feature': 'grade_a',
  'left': {'is_leaf': False,
   'prediction': None,
   'splitting_feature': 'grade_b',
   'left': {'splitting_feature': None,
    'left': None,
    'right': None,
    'is_leaf': True,
    'prediction': -1},
   'right': {'splitting_feature': None,
    'left': None,
    'right': None,
    'is_leaf': True,
    'prediction': -1}},
  'right': {'is_leaf': False,
   'prediction': None,
   'splitting_feature': 'emp_length_< 1 year',
   'left': {'splitting_feature': None,
    'left': None,
    'right': None,
    'is_leaf': True,
    'prediction': 1},
   'right': {'splitting_feature': None,
    'left': None,
    'right': None,
    'is_leaf': True,
    'prediction': -1}}},
 'right': {'is_leaf': False,
  'prediction': None,
  'splitting_feature': 'grade_d',
  'left': {'is_leaf': False,
   'prediction': None,
   'splitting_feature': 'gra

### Build the tree!
Now that all the tests are passing, we will train a tree model on the train_data. Limit the depth to 6 (max_depth = 6) to make sure the algorithm doesn't run for too long. Call this tree my_decision_tree.

Warning: This code block may take 1-2 minutes to learn.

In [30]:
# Make sure to cap the depth at 6 by using max_depth = 6
my_decision_tree = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_a. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_b. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_c. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_d. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_e. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
Re

In [31]:
my_decision_tree

{'is_leaf': False,
 'prediction': None,
 'splitting_feature': 'term_36 months',
 'left': {'is_leaf': False,
  'prediction': None,
  'splitting_feature': 'grade_a',
  'left': {'is_leaf': False,
   'prediction': None,
   'splitting_feature': 'grade_b',
   'left': {'is_leaf': False,
    'prediction': None,
    'splitting_feature': 'grade_c',
    'left': {'is_leaf': False,
     'prediction': None,
     'splitting_feature': 'grade_d',
     'left': {'is_leaf': False,
      'prediction': None,
      'splitting_feature': 'grade_e',
      'left': {'splitting_feature': None,
       'left': None,
       'right': None,
       'is_leaf': True,
       'prediction': -1},
      'right': {'splitting_feature': None,
       'left': None,
       'right': None,
       'is_leaf': True,
       'prediction': -1}},
     'right': {'splitting_feature': None,
      'left': None,
      'right': None,
      'is_leaf': True,
      'prediction': -1}},
    'right': {'splitting_feature': None,
     'left': None,
     '

### Making predictions with a decision tree
As discussed in the lecture, we can make predictions from the decision tree with a simple recursive function. Below, we call this function classify, which takes in a learned tree and a test point x to classify. We include an option annotate that describes the prediction path when set to True.

In [33]:
def classify(tree, x, annotate = False):   
    # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate: 
            print ("At leaf, predicting %s" % tree['prediction'])
        return tree['prediction'] 
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate: 
            print ("Split on %s = %s" % (tree['splitting_feature'], split_feature_value))
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'], x, annotate)

Now, let's consider the first example of the test set and see what my_decision_tree model predicts for this data point.

In [40]:
test_data.iloc[0]

safe_loans                -1
grade_a                    0
grade_b                    0
grade_c                    0
grade_d                    1
grade_e                    0
grade_f                    0
grade_g                    0
term_36 months             0
term_60 months             1
home_ownership_mortgage    0
home_ownership_other       0
home_ownership_own         0
home_ownership_rent        1
emp_length_1 year          0
emp_length_10+ years       0
emp_length_2 years         1
emp_length_3 years         0
emp_length_4 years         0
emp_length_5 years         0
emp_length_6 years         0
emp_length_7 years         0
emp_length_8 years         0
emp_length_9 years         0
emp_length_< 1 year        0
Name: 24, dtype: int64

In [41]:
print ('Predicted class: %s ' % classify(my_decision_tree, test_data.iloc[0]))

Predicted class: -1 


Let's add some annotations to our prediction to see what the prediction path was that lead to this predicted class:

In [42]:
classify(my_decision_tree, test_data.iloc[0], annotate=True)

Split on term_36 months = 0
Split on grade_a = 0
Split on grade_b = 0
Split on grade_c = 0
Split on grade_d = 1
At leaf, predicting -1


-1

In [43]:
classify(small_data_decision_tree, test_data.iloc[0], annotate=True)

Split on term_36 months = 0
Split on grade_a = 0
Split on grade_b = 0
At leaf, predicting -1


-1

**Quiz question: What was the feature that my_decision_tree first split on while making the prediction for test_data[0]?**

**Quiz question: What was the first feature that lead to a right split of test_data[0]?**

**Quiz question: What was the last feature split on before reaching a leaf node for test_data[0]?**

### Evaluating your decision tree
Now, we will write a function to evaluate a decision tree by computing the classification error of the tree on the given dataset.

Again, recall that the classification error is defined as follows:
classification error=# mistakes ÷ # total examples
Now, write a function called evaluate_classification_error that takes in as input:

1. tree (as described above)
2. data (a DataFrame)

This function should return a prediction (class label) for each row in data using the decision tree.

In [46]:
def evaluate_classification_error(tree, data):
    # Apply the classify(tree, x) to each row in your data
    prediction = []
    for i in range(len(data)):
        prediction.append(classify(tree, data.iloc[i]))
        
    # Once you've made the predictions, calculate the classification error and return it
    ## YOUR CODE HERE
    num_of_mistakes = (np.array(prediction) != data[target].values).sum()/float(len(data))
    return num_of_mistakes

Now, let's use this function to evaluate the classification error on the test set.

In [47]:
evaluate_classification_error(my_decision_tree, test_data)

0.37774666092201636

**Quiz Question: Rounded to 2nd decimal point, what is the classification error of my_decision_tree on the test_data?**

### Printing out a decision stump
As discussed in the lecture, we can print out a single decision stump (printing out the entire tree is left as an exercise to the curious reader).

In [48]:
def print_stump(tree, name = 'root'):
    split_name = tree['splitting_feature'] 
    if split_name is None:
        print ("(leaf, label: %s)" % tree['prediction'])
        return None
    split_feature, split_value = split_name.split('_')
    print ('                       %s' % name)
    print ('         |---------------|----------------|')
    print ('         |                                |')
    print ('         |                                |')
    print ('         |                                |')
    print ('  [{0} == 0]               [{0} == 1]    '.format(split_name))
    print ('         |                                |')
    print ('         |                                |')
    print ('         |                                |')
    print ('    (%s)                         (%s)' \
        % (('leaf, label: ' + str(tree['left']['prediction']) if tree['left']['is_leaf'] else 'subtree'),
           ('leaf, label: ' + str(tree['right']['prediction']) if tree['right']['is_leaf'] else 'subtree')))

In [49]:
print_stump(my_decision_tree)

                       root
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [term_36 months == 0]               [term_36 months == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


**Quiz Question: What is the feature that is used for the split at the root node?**

### Exploring the intermediate left subtree
The tree is a recursive dictionary, so we do have access to all the nodes! We can use

my_decision_tree['left'] to go left
my_decision_tree['right'] to go right

In [50]:
print_stump(my_decision_tree['left'], my_decision_tree['splitting_feature'])

                       term_36 months
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade_a == 0]               [grade_a == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


### Exploring the left subtree of the left subtree

In [51]:
print_stump(my_decision_tree['left']['left'], my_decision_tree['left']['splitting_feature'])

                       grade_a
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade_b == 0]               [grade_b == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


**Quiz question: What is the path of the first 3 feature splits considered along the left-most branch of my_decision_tree?**

**Quiz question: What is the path of the first 3 feature splits considered along the right-most branch of my_decision_tree?**

In [53]:
print_stump(my_decision_tree['right'], my_decision_tree['splitting_feature'])

                       term_36 months
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade_d == 0]               [grade_d == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (leaf, label: -1)
